In [2]:
import git
SOURCE_URL = 'https://github.com/CSSEGISandData/COVID-19'
DATA_PATH = '/data'
git.Repo.clone_from(SOURCE_URL, DATA_PATH)

<git.repo.base.Repo 'f:\\data\\.git'>

In [10]:
import sqlite3 as sl
import csv
import os
import re
from re import search

DATA_PATH = "/data"

def GetValuesPart(text):
    return re.sub('\w+[^,]', '?', text)

def ReplaceFields(text, fieldReplaceStorage):
    for key in fieldReplaceStorage:
        text = text.replace(key, fieldReplaceStorage[key])
    return text  

def CreateTableInDB(nameDB, query, queryUS):
    with con:
        cur.execute(query)
        cur.execute(queryUS)

def GetFileNameList(path):
    result = []
    for root, dirs, files in os.walk(path): 
        for file in files: 
            result.append(os.path.join(root,file).replace("\\", "/"))
    return result        
    
def ReplaceInList(row, replaceable, on):
    result = []
    for element in row:
        element = element.replace(replaceable, on)
        result.append(element)
    return result    

def GetFileDateFromName(name):
    return re.search('\d+-\d+-\d+', name).group() 

def AddFileDateFieldToText(headRow):
    if search("\w+", headRow):
        return headRow + ",File_Date"
    else:
      return "File_Date"

def ImportDataFromFileReaderToDB(fileReader, fileDate, insertQuery):
    for row in fileReader:
        row = ReplaceInList(row, "/", "-")
        row.append(fileDate)
        cur.execute(insertQuery, row)

def GetHeadRowFromFileReader(fileReader):
    headRow = ",".join(next(fileReader))
    headRow = ReplaceFields(headRow, fieldReplaceStorage) 
    return headRow

def ImportDataFromFilesToDB(fileNameList, nameTable):
    for name in fileNameList:
        if (".csv" in name):
            with open(name, encoding='utf-8') as r_file:
                fileReader = csv.reader(r_file, delimiter = ",")
                headRow = AddFileDateFieldToText(GetHeadRowFromFileReader(fileReader))
                valuesPart = GetValuesPart(headRow)
                insertQuery = f"INSERT INTO {nameTable} ({headRow}) values({valuesPart}) " 
                fileDate = GetFileDateFromName(name) 
                ImportDataFromFileReaderToDB(fileReader, fileDate, insertQuery)  
                                    

def ImportDataFromFoldersToDB(nameTable, folderPathes):
    try:
        
        fileNameList = GetFileNameList(folderPathes[0])
        ImportDataFromFilesToDB(fileNameList, nameTable)
        fileNameList = GetFileNameList(folderPathes[1])
        ImportDataFromFilesToDB(fileNameList, nameTableUS)               
        con.commit()
    
    finally:
        if cur is not None:
            cur.close()

        if con is not None:
            con.close()

fieldReplaceStorage = {
    "Incidence_Rate": "Incident_Rate",
    "/": "_",
    " ": "_",
    "-": "_"
}

nameTable = "statistics"
nameTableUS = "statisticsUS"
nameDB = "corona_statistics.db"
path1 = DATA_PATH + "\csse_covid_19_data\csse_covid_19_daily_reports"
path2 = DATA_PATH +"\csse_covid_19_data\csse_covid_19_daily_reports_us"
pathList = [path1, path2]

query = """
        CREATE TABLE statistics (
        FIPS                 TEXT,
        Admin2               TEXT,
        Province_State       TEXT,
        Country_Region       TEXT,
        Latitude             REAL,
        Longitude            REAL,
        Last_Update          DATE,
        Lat                  REAL, 
        Long_                REAL,
        Confirmed            INT, 
        Deaths               INT, 
        Recovered            INT, 
        Active               INT, 
        Combined_Key         TEXT,
        Incident_Rate        REAL,
        Case_Fatality_Ratio  REAL,
        File_Date            DATE
    );
    """  
queryUS = """
        CREATE TABLE statisticsUS (
        FIPS                 TEXT,
        Province_State       TEXT,
        Country_Region       TEXT,
        Last_Update          DATE,
        Lat                  REAL, 
        Long_                REAL,
        Confirmed            INT, 
        Deaths               INT, 
        Recovered            INT, 
        Active               INT, 
        Incident_Rate        REAL,
        Case_Fatality_Ratio  REAL,
        Total_Test_Results   REAL,
        People_Hospitalized  REAL,
        UID                  REAL,
        ISO3                 TEXT,
        Testing_Rate         REAL,
        Hospitalization_Rate REAL,
        People_Tested        REAL,
        Mortality_Rate       REAL,
        Cases_28_Days        REAL,
        Deaths_28_Days       REAL,
        File_Date            DATE
    );
    """

In [7]:
con = sl.connect(nameDB)  
cur = con.cursor()  

In [8]:
CreateTableInDB(nameDB, query, queryUS)

In [9]:
ImportDataFromFoldersToDB(nameTable, pathList)